![MADE](resources/made.jpg)

# Академия MADE


# Семинар 13: GAN для генерации лиц

Сегодня мы займемся задачей генерации лиц с помощью генеративно-состязательных сетей (GAN). Мы не будем применять никаких эвристик, связанных именно с лицами, поэтому полученный в итоге пайплайн можно будет использовать для генерации объектов иной природы, подставив нужный датасет (ну и, скорее всего, подправив какие-то из гиперпараметров).

#### **План**:
1. **GAN & картинки: Deep Convolutional GAN (DCGAN)**
2. **(Baseline) DCGAN & BCELoss.**
3. **(Wasserstein) Wasserstein distance & Gradient Penalty.**
4. **(Advanced) Spectral Normalization, Self-Attention, TTUR.**
5. **Анализ проблем и что делать дальше**

## 1. GAN & картинки

### 1.1. Tiny recap

![Overview](resources/gan_overview.png)

[отсюда](https://www.kdnuggets.com/2017/01/generative-adversarial-networks-hot-topic-machine-learning.html)

В базовом случае ([Goodfellow et al, 2014](https://papers.nips.cc/paper/5423-generative-adversarial-nets.pdf)) схема обучения GAN такова:
- Есть две модели, генератор $G$ и дискриминатор $D$.
- Генератор: 
  - На вход получает вектор шума $z$,
  - На выходе показывает объект (например, картинку) $G(z)$.

- Дискриминатор: 
  - На вход получает либо настоящий, либо сгенерированный объект ($x$, $G(z)$),
  - На выходе отдает уверенность в том, что объект - настоящий ($D(x)$, $D(G(z))$).

Обучение генератора и дискриминатора производится отдельными шагами:
- "Ошибка дискриминатора" = **BCELoss**
- На шаге обучения генератора ошибка дискриминатора **максимизируется** (градиентный подъем),
- На шаге обучения дискриминатора ошибка дискриминатора **минимизируется** (good old градиентный спуск).

На лекциях было показано, что при такой схеме обучения должно происходить "сближение" двух распределений - "настоящего" $p_{data}(x)$ и "сгенерированного" $p_{g}(x)$ - с дивергенцией Йенсена-Шэннона в качестве критерия близости.

### 1.2. Засовываем картинки в GAN

Авторы оригинальной статьи использовали в экспериментах только полносвязные сети, в том числе и для генерации изображений (в низком разрешении).
Как мы знаем, там, где нужна работа с картинками, ~~полносвязным сетям места нет~~ обычно более эффективными оказываются сверточные сети. Одной из первых публикаций по теме использования сверточных сетей для генерации изображений с помощью GAN была статья ["Unsupervised Representation Learning With Deep Convolutional Generative Adversarial Networks"](https://arxiv.org/pdf/1511.06434.pdf). Основные тезисы:
* Не использовать слои "грубого" изменения размера карт активаций (`Pooling`, `Upsampling`); использовать сверточные слои (со `stride`> 1) и слои с транспонированными свертками,
* Использовать `BatchNorm` в обеих моделях,
* Не использовать полносвязные слои вообще,
* В генераторе использовать `ReLU` и `Tanh` (в конце),
* В дискриминаторе использовать `LeakyReLU`.

![dcgan](resources/dcgan.png)

Кроме того, один из соавторов статьи про `DCGAN` и ключевая фигура в разработке `PyTorch` Soumith Chintala выложил [свои рекомендации](https://github.com/soumith/ganhacks) по обучению GAN. Некоторые из них мы используем в своих экспериментах, а именно:
- Не использовать "смешанные" батчи (из настоящих и сгенерированных изображений), делать инференс по-отдельности,
- Использовать "soft labels" (`0+eps` вместо `0`, `1.0-eps` вместо `1`).

Далее соберем архитектуру, подобную `DCGAN`, обучим ее с небольшими изменениями и посмотрим, что из этого получится.

## 2. (Baseline) DCGAN & BCELoss

При установке `TRAIN` = `False` вместо обучения будут подгружаться веса модели и ожидаемые результаты.

In [ ]:
TRAIN = False

Реализуем:
1. Подгрузку данных
2. Классы для дискриминатора и генератора
3. Функцию для обучения

In [ ]:
from utils import get_device, reproduce, show_data_batch

import torch
from torch import nn
from torch import optim
from torch import autograd

from torch.nn import Parameter as P
import torch.nn.functional as F

import torchvision.datasets
import torchvision.transforms as transforms
import torchvision.utils as vutils


import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

import numpy as np
import pickle

In [ ]:
reproduce()

### 2.1. Данные

Надо скачать [выровненные изображения лиц](https://drive.google.com/file/d/0B7EVK8r0v71pZjFTYXZWM3FlRnM/view?usp=sharing) из датасета [CelebA](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html). Затем распаковать и положить в папку `DATA_ROOT`.

*Параметры и данные из п.2.1. будем использовать во всех дальнейших экспериментах без изменений.* 

In [ ]:
DATA_ROOT = "./data/"
IMAGE_SIZE = 64
BATCH_SIZE = 256
NUM_WORKERS = 16
NUM_TO_SHOW = 64

Класс `ImageFolder` умеет доставать картинки из подпапок корневой директории, при этом считая отдельный папки отдельными классами. Нам метки классов не понадобятся, не забудем учесть это при получении батчей из загрузчика данных.

In [ ]:
dataset = torchvision.datasets.ImageFolder(root=DATA_ROOT,
                                           transform=transforms.Compose([
                                               transforms.Resize(IMAGE_SIZE),
                                               transforms.CenterCrop(IMAGE_SIZE),
                                               transforms.ToTensor(),
                                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                           ]))

dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS,
                                         drop_last=True, pin_memory=True)

batch = next(iter(dataloader))
show_data_batch(batch, max_images=NUM_TO_SHOW)

### 2.2. Модели

Зададим гиперпараметры моделей генератора и дискриминатора.
* `LATENT_DIM`: размерность (длина) вектора шума, из которого мы будем получать лица с помощью генератора,
* `IMAGE_CHANNELS`: число каналов в изображениях (если захочется генерировать grayscale-изображения, нужно поставить `=1`),
* `*_BASE_FEATURES`: параметры, определяющие начальнуе глубину сверточных слоев в обеих моделях.

In [ ]:
LATENT_DIM = 100
IMAGE_CHANNELS = 3
DISCRIMINATOR_BASE_FEATURES = 64
GENERATOR_BASE_FEATURES = 64

#### 2.2.1. Дискриминатор

Начнем с дискриминатора. Учтя рекомендациям авторов `DCGAN`, его структуру сделаем следующей:
* На вход подается изображение с заданным числом каналов (`input_channels`);
* Тело состоит из последовательных блоков вида `Conv2d - BN2d  - LeakyReLU`; в первом блоке опущен `BN2d` (**почему?**), в последнем есть только `Conv2d`;
* Сверточные слои сделаем со `stride=2`, т.к. мы не хотим использовать `MaxPool2d`; `kernel_size=4`, `padding=1`;
* Добавим параметр `with_sigmoid` на будущее (понадобится, когда перейдем от кросс-энтропии к другому критерию).

In [ ]:
class DiscriminatorBasic(nn.Module):
    
    def __init__(self, 
                 input_channels=IMAGE_CHANNELS, 
                 base_num_features=DISCRIMINATOR_BASE_FEATURES, 
                 with_sigmoid=True):
        super(DiscriminatorBasic, self).__init__()
        layers = [
            # input size: input_channels x 64 x 64
            
            nn.Conv2d(input_channels, base_num_features, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size: base_num_features x 32 x 32

            nn.Conv2d(base_num_features, base_num_features * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(base_num_features * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size: (base_num_features * 2) x 16 x 16

            nn.Conv2d(base_num_features * 2, base_num_features * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(base_num_features * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size: (base_num_features * 4) x 8 x 8

            nn.Conv2d(base_num_features * 4, base_num_features * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(base_num_features * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size: (base_num_features * 8) x 4 x 4

            nn.Conv2d(base_num_features * 8, 1, 4, 1, 0, bias=False),
            # state size: 1 x 1 x 1
        ]
        
        if with_sigmoid:
            layers.append(nn.Sigmoid())
        
        self.main = nn.Sequential(*layers)

    def forward(self, inputs):
        return self.main(inputs)

In [ ]:
discriminator = DiscriminatorBasic()
print(discriminator)

Модели рассчитаны на работу с изображениями `64x64` (размер уменьшается в 64 раза); проверим, что на выходе получается одно-единственное число (уверенность дискриминатора в том, что пример - реальный):

In [ ]:
x = torch.randn(4, IMAGE_CHANNELS, IMAGE_SIZE, IMAGE_SIZE)
y = discriminator(x)
assert y.size() == (4, 1, 1, 1), y.size()

#### 2.2.2. Генератор

С генератором чуть сложнее: на вход он получает вектор шума `z` длины `LATENT_DIM`, но в следующем виде: `LATENT_DIM x 1 x 1`.
В отличие от дискриминатора, здесь используем транспонированные свертки, активации `ReLU` в середине и `Tanh` в самом конце.

In [ ]:
class GeneratorBasic(nn.Module):
    
    def __init__(self, 
                 input_channels=LATENT_DIM, 
                 base_num_features=GENERATOR_BASE_FEATURES, 
                 output_channels=IMAGE_CHANNELS):
        super(GeneratorBasic, self).__init__()
        layers = [
            # input is Z, going into a convolution
            
            nn.ConvTranspose2d(input_channels, base_num_features * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(base_num_features * 8),
            nn.ReLU(inplace=True),
            # state size: (base_num_features * 8) x 4 x 4

            nn.ConvTranspose2d(base_num_features * 8, base_num_features * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(base_num_features * 4),
            nn.ReLU(inplace=True),
            # state size: (base_num_features * 4) x 8 x 8

            nn.ConvTranspose2d(base_num_features * 4, base_num_features * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(base_num_features * 2),
            nn.ReLU(inplace=True),
            # state size: (base_num_features * 2) x 16 x 16

            nn.ConvTranspose2d(base_num_features * 2, base_num_features, 4, 2, 1, bias=False),
            nn.BatchNorm2d(base_num_features),
            nn.ReLU(inplace=True),
            # state size: (base_num_features) x 32 x 32

            nn.ConvTranspose2d(base_num_features, output_channels, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size: output_channels x 64 x 64
        ]

        self.main = nn.Sequential(*layers)

    def forward(self, inputs):
        return self.main(inputs)

In [ ]:
generator = GeneratorBasic()
print(generator)

Генератор тоже заточен под один размер изображений - `64x64`. 

In [ ]:
x = torch.randn(4, LATENT_DIM, 1, 1)
y = generator(x)
assert y.size() == (4, IMAGE_CHANNELS, IMAGE_SIZE, IMAGE_SIZE), y.size()

Используем также функцию для ручной инициализации весов наших моделей (**как инициализируются веса по умолчанию?**):

In [ ]:
def weights_init(m, scale=0.02):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight, 0.0, scale)
    elif classname.find('BatchNorm') != -1:
        torch.nn.init.normal_(m.weight, 1.0, scale)
        torch.nn.init.zeros_(m.bias)

Зададим фиксированный вектор шума, чтобы отслеживать успехи генератора по мере обучения GAN:

In [ ]:
device = get_device()
print(device)

FIXED_NOISE = torch.randn(NUM_TO_SHOW, LATENT_DIM, 1, 1, device=device)
print(FIXED_NOISE.size())

### 2.3. Код для обучения

Используем `smooth labels` и [рекомендованные](https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html) параметры оптимизатора (`Adam`):

In [ ]:
REAL_LABEL = 0.9  # better use 1 - random(0.0, 0.1)
FAKE_LABEL = 0.1  # better use random(0.0, 0.1)

Перейдем к написанию функции, которая будет получать на вход модели/оптимизаторы/данные/прочие параметры и выполнять 1 эпоху обучения. Эту функцию мы напишем несколько раз по мере усложнения пайплайна обучения.

Напомним алгоритм обучения GAN в [первозданном](https://papers.nips.cc/paper/5423-generative-adversarial-nets.pdf) виде:

![Algo](resources/gan_algo.png)

In [ ]:
def train_epoch_basic(generator, discriminator, 
                      optimizer_generator, optimizer_discriminator,
                      dataloader, epoch, num_epochs, device,
                      criterion):
    generator.to(device)
    discriminator.to(device)

    # Logging routine
    generator_loss_list, discriminator_loss_list = [], []
    discriminator_prob_real_list, discriminator_prob_fake_before_list, discriminator_prob_fake_after_list = [], [], []
    generator_images_list = []

    for i, batch in enumerate(dataloader):

        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ############################
        discriminator.zero_grad()
        
        ## Train with all-real batch
        # Forward pass real batch through D
        real_batch, _ = batch
        output = discriminator(real_batch.to(device)).view(-1)
        
        # Calculate loss on all-real batch
        label = torch.full((BATCH_SIZE,), REAL_LABEL, device=device)
        discriminator_loss_real = criterion(output, label)
        
        # Calculate gradients for D in backward pass
        discriminator_loss_real.backward()
        
        # Save for logging
        discriminator_prob_real = output.mean().item()
        discriminator_prob_real_list.append(discriminator_prob_real)

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(BATCH_SIZE, LATENT_DIM, 1, 1, device=device)
        
        # Generate fake image batch with G
        fake_batch = generator(noise)
        
        # Classify all fake batch with D
        output = discriminator(fake_batch.detach()).view(-1)  # Detach!
        
        # Save for logging
        discriminator_prob_fake_before = output.mean().item()
        discriminator_prob_fake_before_list.append(discriminator_prob_fake_before)
        
        # Calculate D's loss on the all-fake batch
        label.fill_(FAKE_LABEL)
        discriminator_loss_fake = criterion(output, label)
        
        # Calculate the gradients for this batch
        discriminator_loss_fake.backward()
        
        discriminator_loss = discriminator_loss_real + discriminator_loss_fake
        
        # Update D
        optimizer_discriminator.step()
        
        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ############################
        generator.zero_grad()
        
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = discriminator(fake_batch).view(-1)
                
        # Save for logging
        discriminator_prob_fake_after = output.mean().item()
        discriminator_prob_fake_after_list.append(discriminator_prob_fake_after)
        
        # Calculate G's loss based on this output
        label.fill_(REAL_LABEL)  # fake labels are real for generator cost
        generator_loss = criterion(output, label)
        
        # Calculate gradients for G
        generator_loss.backward()
        
        # Update G
        optimizer_generator.step()

        # Output training stats
        if i % 500 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     discriminator_loss.item(), generator_loss.item(), discriminator_prob_real,
                     discriminator_prob_fake_before, discriminator_prob_fake_after))

        # Save Losses for plotting later
        generator_loss_list.append(generator_loss.item())
        discriminator_loss_list.append(discriminator_loss.item())

        # Check how the generator is doing by saving G's output on fixed_noise
        if i == len(dataloader) - 1:
            generator.eval()
            with torch.no_grad():
                fake = generator(FIXED_NOISE).detach().cpu()
            generator_images_list.append(vutils.make_grid(fake, padding=2, normalize=True))
            generator.train()

    return {"generator_loss_list": generator_loss_list,
            "discriminator_loss_list": discriminator_loss_list,
            "discriminator_prob_real_list": discriminator_prob_real_list,
            "discriminator_prob_fake_before_list": discriminator_prob_fake_before_list,
            "discriminator_prob_fake_after_list": discriminator_prob_fake_after_list,
            "generator_images_list": generator_images_list}

### 2.4. Обучение

In [ ]:
LR = 0.0002
BETAS = [0.5, 0.999]

In [ ]:
generator = GeneratorBasic()
generator.apply(weights_init)
optimizer_generator = optim.Adam(generator.parameters(), lr=LR, betas=BETAS, amsgrad=True)

discriminator = DiscriminatorBasic()
discriminator.apply(weights_init)
optimizer_discriminator = optim.Adam(discriminator.parameters(), lr=LR, betas=BETAS, amsgrad=True)

criterion = nn.BCELoss()

In [ ]:
results = {key: [] for key in ["generator_loss_list", 
                               "discriminator_loss_list", 
                               "discriminator_prob_real_list", 
                               "discriminator_prob_fake_before_list", "discriminator_prob_fake_after_list",
                               "generator_images_list"]}

In [ ]:
if TRAIN:
    NUM_EPOCHS = 50
    for epoch in range(NUM_EPOCHS):
        epoch_results = train_epoch_basic(generator, discriminator, 
                                          optimizer_generator, optimizer_discriminator,
                                          dataloader, epoch, NUM_EPOCHS, device,
                                          criterion)
        for key in results:
            results[key].extend(epoch_results[key])
            
    with open("./cached/results_basic.pkl", "wb") as fp:
        pickle.dump(results, fp)
    with open("./weights/checkpoint_basic.pth.tar", "wb") as fp:
        torch.save({"generator": generator.state_dict(),
                    "discriminator": discriminator.state_dict()}, fp)
else:
    with open("./cached/training_basic.txt", "rt") as fp:
        for line in fp:
            print(line.strip())
    with open("./cached/results_basic.pkl", "rb") as fp:
        results = pickle.load(fp)
    with open("./weights/checkpoint_basic.pth.tar", "rb") as fp:
        states = torch.load(fp, map_location="cpu")
        generator.load_state_dict(states["generator"])
        discriminator.load_state_dict(states["discriminator"])

In [ ]:
plt.figure(figsize=(16,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(results["generator_loss_list"], label="G", alpha=0.5)
plt.plot(results["discriminator_loss_list"], label="D", alpha=0.5)
plt.xlabel("iterations")
plt.ylabel("Loss")
# plt.yscale("log")
plt.legend()
plt.show()

In [ ]:
#%%capture
fig = plt.figure(figsize=(16,16))
plt.axis("off")
imgs = [[plt.imshow(np.transpose(i, (1, 2, 0)), animated=True)] for i in results["generator_images_list"][::5]]
ani = animation.ArtistAnimation(fig, imgs, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

## 3. (Wasserstein) Wasserstein distance & Gradient Penalty

Вспомним, что использованная нами только что схема обучения (с кросс-энтропией) была подвергнута справедливой критике [не](https://arxiv.org/pdf/1701.07875) [раз](https://arxiv.org/pdf/1611.04076). В качестве альтернативы $D_{js}$ как метрики близости двух распределений предлагается, например, использовать [аппроксимацию расстояния Васерштейна](https://arxiv.org/abs/1701.07875) (оно же `Earth Mover Distance`):

![Wdist](resources/wasserstein_loss.png)

Это, в свою очередь, приводит к следующему алгоритму обучения:

![Algo](resources/wgan_algo.png)

Важно отметить, что для работы в этой схеме появляются ограничения на функцию, вычисляемую дискриминатором: она должна быть ограниченной по Липшицу. Для этого в оригинальной статье предлагается довольно грубый метод - усечения весов дискриминатора (см. п.7 алгоритма выше). В [статье](https://arxiv.org/pdf/1704.00028.pdf), развивающей идеи Wasserstein GAN, предложен альтернативный подход к ограничиванию константы Липшица для дискриминатора - это непосредственное добавление в функцию потерь нормы разности градиентов весов дискриминатора и единицы. Мы попробуем только второй подход.

### 3.1. Wasserstein GAN + Gradient Penalty

Этот подход описан в статье [Improved Training of Wasserstein GANs](https://arxiv.org/pdf/1704.00028.pdf), где утверждается, что при использовании Gradient penalty генератор лучше аппроксимирует мультимодальные распределения, а обучение становится более стабильным.
Итоговый критерий выглядит так:

![gp](resources/wgan_gradient_penalty.png)

Таким образом, нам понадобится дополнительно реализовать функцию для вычисления нормы градиентов весов дискриминатора и поместить ее в соответствующий лосс.
Взято [отсюда](https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/wgan_gp/wgan_gp.py). 

Обратите внимание, на каких примерах считаются нормы градиентов - это результаты линейной интерполяции между сгенерированными и реальными изображениями.

In [ ]:
def compute_gradient_penalty(discriminator, real_batch, fake_batch, device):
    # Random weight term for interpolation between real and fake samples
    alpha = torch.Tensor(np.random.random((real_batch.size(0), 1, 1, 1))).to(device)
    
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_batch + ((1 - alpha) * fake_batch)).requires_grad_(True)
    d_interpolates = discriminator(interpolates).cpu()
    
    # Get gradient w.r.t. interpolates
    fake = torch.ones(real_batch.shape[0], 1, 1, 1).float().requires_grad_(False)
    gradients = autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

In [ ]:
def train_epoch_wasserstein(generator, discriminator, 
                            optimizer_generator, optimizer_discriminator,
                            dataloader, epoch, num_epochs, device,
                            n_critic, gradient_penalty_lambda):
    generator.to(device)
    discriminator.to(device)

    generator_loss_list, discriminator_loss_list = [], []
    discriminator_gradient_penalty_list = []
    generator_images_list = []

    for i, batch in enumerate(dataloader):

        ############################
        # (1) Update D network
        ############################
        discriminator.zero_grad()
        
        real_batch = batch[0].to(device)
        output_real = discriminator(real_batch)
        
        noise = torch.randn(BATCH_SIZE, LATENT_DIM, 1, 1, device=device)
        fake_batch = generator(noise)
        output_fake = discriminator(fake_batch)
        
        discriminator_loss = - torch.mean(output_real) + torch.mean(output_fake)
        discriminator_loss_list.append(discriminator_loss.item())
        
        if gradient_penalty_lambda > 0:
            gradient_penalty = compute_gradient_penalty(discriminator, real_batch, fake_batch, device)
            discriminator_gradient_penalty_list.append(gradient_penalty.item())

            discriminator_loss += gradient_penalty_lambda * gradient_penalty
        else:
            gradient_penalty = torch.zeros(1)
        
        discriminator_loss.backward()
        optimizer_discriminator.step()
        

        if i % n_critic == 0:
                
        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ############################
            generator.zero_grad()
            
            noise = torch.randn(BATCH_SIZE, LATENT_DIM, 1, 1, device=device)
            fake_batch = generator(noise)
            output_fake = discriminator(fake_batch)
            
            output = discriminator(fake_batch)
            generator_loss = - torch.mean(output_fake)
            generator_loss.backward()
            optimizer_generator.step()
        
        generator_loss_list.append(generator_loss.item())

        # Output training stats
        if i % 500 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tGP: %.4f' %
                  (epoch, num_epochs, i, len(dataloader), 
                   discriminator_loss.item(), generator_loss.item(), gradient_penalty.item()))

        # Check how the generator is doing by saving G's output on fixed_noise
        if i == len(dataloader) - 1:
            with torch.no_grad():
                fake = generator(FIXED_NOISE).detach().cpu()
            generator_images_list.append(vutils.make_grid(fake, padding=2, normalize=True))

    return {"generator_loss_list": generator_loss_list,
            "discriminator_loss_list": discriminator_loss_list,
            "discriminator_gradient_penalty_list": discriminator_gradient_penalty_list,
            "generator_images_list": generator_images_list}

### 3.2. Обучение

In [ ]:
GRADIENT_PENALTY_LAMBDA = 10

N_CRITIC = 5
LR = 0.0001

Поскольку в `Wasserstein GAN` дискриминатор не является бинарным классификатором (а служит функцией, определяющей расстояние между двумя распределениями), оборачивать его выход нелинейностью не нужно.

In [ ]:
generator = GeneratorBasic()
generator.apply(weights_init)
optimizer_generator = optim.RMSprop(generator.parameters(), lr=LR)


discriminator = DiscriminatorBasic(with_sigmoid=False)
discriminator.apply(weights_init)
optimizer_discriminator = optim.RMSprop(discriminator.parameters(), lr=LR)

criterion = None

In [ ]:
results = {key: [] for key in ["generator_loss_list", 
                               "discriminator_loss_list", 
                               "discriminator_gradient_penalty_list",
                               "generator_images_list"]}

In [ ]:
if TRAIN:
    NUM_EPOCHS = 50
    for epoch in range(NUM_EPOCHS):
        epoch_results = train_epoch_wasserstein(generator, discriminator,
                                                optimizer_generator, optimizer_discriminator,
                                                dataloader, epoch, NUM_EPOCHS, device,
                                                N_CRITIC, GRADIENT_PENALTY_LAMBDA)
        for key in results:
            results[key].extend(epoch_results[key])
        
    with open("./cached/results_wasserstein.pkl", "wb") as fp:
        pickle.dump(results, fp)
    with open("./weights/checkpoint_wasserstein.pth.tar", "wb") as fp:
        torch.save({"generator": generator.state_dict(),
                    "discriminator": discriminator.state_dict()}, fp)
else:
    with open("./cached/training_wasserstein.txt", "rt") as fp:
        for line in fp:
            print(line.strip())
    with open("./cached/results_wasserstein.pkl", "rb") as fp:
        results = pickle.load(fp)
    with open("./weights/checkpoint_wasserstein.pth.tar", "rb") as fp:
        states = torch.load(fp, map_location="cpu")
        generator.load_state_dict(states["generator"])
        discriminator.load_state_dict(states["discriminator"])

In [ ]:
plt.figure(figsize=(16,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(results["generator_loss_list"], label="G", alpha=0.5)
plt.plot(results["discriminator_loss_list"], label="D", alpha=0.5)
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
#%%capture
fig = plt.figure(figsize=(16,16))
plt.axis("off")
imgs = [[plt.imshow(np.transpose(i, (1, 2, 0)), animated=True)] for i in results["generator_images_list"][::5]]
ani = animation.ArtistAnimation(fig, imgs, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

## 4. (Advanced) Spectral Normalization, Self-Attention, TTUR

Расширим теперь возможности наши модели, следуя некоторым идеям из статьи [SAGAN](https://arxiv.org/pdf/1805.08318.pdf):
* Во-первых, добавим в модели слои `Self-Attention`,
* Во-вторых, добавим в модели поддержку спектральной нормализации весов,
* В-третьих, будем использовать разные `LR` для генератора и дискриминатора (но обновлять обе модели с одной частотой)

### 4.1. Self-Attention

Авторы отмечают, что очень важным аспектом генерации изображений является рецептивное поле нейронов в сверточных слоях. В самом деле, поскольку архитектуры наподобие `DCGAN` не используют полносвязные слои (по отдельным причинам), то может оказаться так, что нейроны последних слоев дискриминатора будут видеть только часть изображения. Это, в свою очередь, может позволять генератору выдавать изображения собак с 5 ногами и все в таком духе.

В качестве решения они предлагают использовать слой `Self-Attention`, который позволяет "передавать" конкретному нейрону, видящему только "свою" область, информацию обо всех других областях. При этом нейрон сам "решает", насколько важным для него является каждая конкретная область, поскольку используется механизм взвешивания.

![SA](resources/self_attention.png)

Реализацию возьмем из [репозитория](https://github.com/ajbrock/BigGAN-PyTorch/blob/master/layers.py) BigGAN.

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, ch, with_sn=False):
        super(SelfAttention, self).__init__()
        # Channel multiplier
        self.ch = ch
        self.conv2d = SNConv2d if with_sn else nn.Conv2d
        self.theta = self.conv2d(self.ch, self.ch // 8, kernel_size=1, padding=0, bias=False)
        self.phi = self.conv2d(self.ch, self.ch // 8, kernel_size=1, padding=0, bias=False)
        self.g = self.conv2d(self.ch, self.ch // 2, kernel_size=1, padding=0, bias=False)
        self.o = self.conv2d(self.ch // 2, self.ch, kernel_size=1, padding=0, bias=False)
        # Learnable gain parameter
        self.gamma = P(torch.tensor(0.), requires_grad=True)

    def forward(self, x, y=None):
        # Apply convs
        theta = self.theta(x)
        phi = F.max_pool2d(self.phi(x), [2, 2])
        g = F.max_pool2d(self.g(x), [2, 2])
        # Perform reshapes
        theta = theta.view(-1, self.ch // 8, x.shape[2] * x.shape[3])
        phi = phi.view(-1, self.ch // 8, x.shape[2] * x.shape[3] // 4)
        g = g.view(-1, self.ch // 2, x.shape[2] * x.shape[3] // 4)
        # Matmul and softmax to get attention maps
        beta = F.softmax(torch.bmm(theta.transpose(1, 2), phi), -1)
        # Attention map times g path
        o = self.o(torch.bmm(g, beta.transpose(1, 2)).view(-1, self.ch // 2, x.shape[2], x.shape[3]))
        return self.gamma * o + x

### 4.2. Spectral Normalization

В работе по `Wasserstein GAN` было показано, что для его стабильной сходимости необходимо накладывать ограничения на дискриминатор. Впрочем, оказалось, что ограниченность дискриминатора помогает и в более общих случаях (при других функционалах). Авторы статьи [Spectral Normalization for Generative Adversarial Networks](https://arxiv.org/abs/1802.05957) предложили применять для ограничения дискриминатора спектральную нормализацию весов, что, по задумке, должно приводить к константе Липшица ~ 1. Но если в случае с методом `Gradient Penalty` (использованном нами выше) ограничивание каких-то данных (на которых считается градиент и его норма), то спектральная нормализация данных не требует. 

В статье `SAGAN` авторы использовали эту нормализацию и в генераторе тоже. Мы попробуем вариант с нормализацией только в дискриминатор, самостоятельно можете попробовать и другие.

In [ ]:
from spectral import SNConv2d, SNConvTranspose2d  # тоже из BigGAN

In [ ]:
class DiscriminatorAdvanced(nn.Module):
    
    def __init__(self, input_channels=IMAGE_CHANNELS, base_num_features=DISCRIMINATOR_BASE_FEATURES, with_sn=False):
        super(DiscriminatorAdvanced, self).__init__()
        conv2d = SNConv2d if with_sn else nn.Conv2d
        layers = [
            # input size: input_channels x 64 x 64
            
            conv2d(input_channels, base_num_features, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size: base_num_features x 32 x 32

            conv2d(base_num_features, base_num_features * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(base_num_features * 2) if not with_sn else nn.Identity(),
            nn.LeakyReLU(0.2, inplace=True),
            SelfAttention(base_num_features * 2, with_sn=with_sn),
            # state size: (base_num_features * 2) x 16 x 16

            conv2d(base_num_features * 2, base_num_features * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(base_num_features * 4) if not with_sn else nn.Identity(),
            nn.LeakyReLU(0.2, inplace=True),
            # state size: (base_num_features * 4) x 8 x 8

            conv2d(base_num_features * 4, base_num_features * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(base_num_features * 8) if not with_sn else nn.Identity(),
            nn.LeakyReLU(0.2, inplace=True),
            SelfAttention(base_num_features * 8, with_sn=with_sn),
            # state size: (base_num_features * 8) x 4 x 4

            conv2d(base_num_features * 8, 1, 4, 1, 0, bias=False),
            # state size: 1 x 4 x 4
        ]
        
        self.main = nn.Sequential(*layers)

    def forward(self, inputs):
        return self.main(inputs)

In [ ]:
discriminator = DiscriminatorAdvanced(with_sn=True)
print(discriminator)

In [ ]:
x = torch.randn(4, IMAGE_CHANNELS, IMAGE_SIZE, IMAGE_SIZE)
y = discriminator(x)
assert y.size() == (4, 1, 1, 1), y.size()

In [ ]:
class GeneratorAdvanced(nn.Module):
    
    def __init__(self, input_channels=LATENT_DIM, base_num_features=GENERATOR_BASE_FEATURES, output_channels=IMAGE_CHANNELS, with_sn=False):
        super(GeneratorAdvanced, self).__init__()
        self.conv2d_t = SNConvTranspose2d if with_sn else nn.ConvTranspose2d
        layers = [
            # input is Z, going into a convolution
            
            self.conv2d_t(input_channels, base_num_features * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(base_num_features * 8) if not with_sn else nn.Identity(),
            nn.ReLU(inplace=True),
            # state size: (base_num_features * 8) x 4 x 4

            self.conv2d_t(base_num_features * 8, base_num_features * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(base_num_features * 4) if not with_sn else nn.Identity(),
            nn.ReLU(inplace=True),
            SelfAttention(base_num_features * 4, with_sn=with_sn),
            # state size: (base_num_features * 4) x 8 x 8

            self.conv2d_t(base_num_features * 4, base_num_features * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(base_num_features * 2) if not with_sn else nn.Identity(),
            nn.ReLU(inplace=True),
            # state size: (base_num_features * 2) x 16 x 16

            self.conv2d_t(base_num_features * 2, base_num_features, 4, 2, 1, bias=False),
            nn.BatchNorm2d(base_num_features) if not with_sn else nn.Identity(),
            nn.ReLU(inplace=True),
            SelfAttention(base_num_features, with_sn=with_sn),
            # state size: (base_num_features) x 32 x 32

            self.conv2d_t(base_num_features, output_channels, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size: output_channels x 64 x 64
        ]

        self.main = nn.Sequential(*layers)

    def forward(self, inputs):
        return self.main(inputs)

In [ ]:
generator = GeneratorAdvanced(with_sn=False)
print(generator)

In [ ]:
x = torch.randn(4, LATENT_DIM, 1, 1)
y = generator(x)
assert y.size() == (4, IMAGE_CHANNELS, IMAGE_SIZE, IMAGE_SIZE), y.size()

### 4.3. Обучение

Перейдем к схеме, в которой дискриминатор и генератор обучаются с одной частотой (то есть `N_CRITIC = 1`), но выставим разные значения `LR`.

In [ ]:
N_CRITIC = 1
DISCRIMINATOR_LR = 0.0005
GENERATOR_LR = 0.0001
BETAS = [0.0, 0.999]
GRADIENT_PENALTY_LAMBDA = 0

In [ ]:
generator = GeneratorAdvanced(with_sn=False)
generator.apply(weights_init)
optimizer_generator = optim.Adam(generator.parameters(), lr=LR, betas=BETAS, amsgrad=True)

discriminator = DiscriminatorAdvanced(with_sn=True)
discriminator.apply(weights_init)
optimizer_discriminator = optim.Adam(discriminator.parameters(), lr=LR, betas=BETAS, amsgrad=True)

criterion = None

In [ ]:
results = {key: [] for key in ["generator_loss_list", 
                               "discriminator_loss_list", 
                               "discriminator_gradient_penalty_list",
                               "generator_images_list"]}

In [ ]:
if TRAIN:
    NUM_EPOCHS = 50
    for epoch in range(NUM_EPOCHS):
        epoch_results = train_epoch_wasserstein(generator, discriminator,
                                                optimizer_generator, optimizer_discriminator,
                                                dataloader, epoch, NUM_EPOCHS, device,
                                                N_CRITIC, GRADIENT_PENALTY_LAMBDA)
        for key in results:
            results[key].extend(epoch_results[key])
        
    with open("./cached/results_advanced.pkl", "wb") as fp:
        pickle.dump(results, fp)
    with open("./weights/checkpoint_advanced.pth.tar", "wb") as fp:
        torch.save({"generator": generator.state_dict(),
                    "discriminator": discriminator.state_dict()}, fp)
else:
    with open("./cached/training_advanced.txt", "rt") as fp:
        for line in fp:
            print(line.strip())
    with open("./cached/results_advanced.pkl", "rb") as fp:
        results = pickle.load(fp)
    with open("./weights/checkpoint_advanced.pth.tar", "rb") as fp:
        states = torch.load(fp, map_location="cpu")
        generator.load_state_dict(states["generator"])
        discriminator.load_state_dict(states["discriminator"])

In [ ]:
plt.figure(figsize=(16,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(results["generator_loss_list"], label="G", alpha=0.5)
plt.plot(results["discriminator_loss_list"], label="D", alpha=0.5)
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
#%%capture
fig = plt.figure(figsize=(16,16))
plt.axis("off")
imgs = [[plt.imshow(np.transpose(i, (1, 2, 0)), animated=True)] for i in results["generator_images_list"][::5]]
ani = animation.ArtistAnimation(fig, imgs, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

## 5. Что делать дальше

Мы рассмотрели базовый подход к генерации изображений с помощью GAN и немного расширили его, заменив критерий обучения и добавив несколько компонентов для улучшения сходимости моделей. Однако, любая из упомянутых в семинаре статей (и тем более из неупомянутых) содержит еще много идей, как сделать генерацию лучше. В конце страницы вы найдете список статей (субъективный), к которым можно обратиться за идеями. А пока коротко перечислим, что можно сделать в продолжение семинара.

**Во-первых, исследовать проблему коллапсирования мод в полученных моделях.** Невооруженным глазом видно, что некоторые из сгенерированных изображений очень похожи друг на друга. Попробуйте сгенерировать несколько лиц и отыскать для них ближайшие по обучающей выборке. 

**Во-вторых, исследовать "гладкость" представлений в генераторе.** Сгенерируйте пару случайных векторов $(z_1, z_2)$ и, сделав линейную интерполяцию между ними , насэмплируйте несколько новых векторов $z'=\alpha z_1 + (1 - \alpha) z_2$. Пропустите векторы через генератор и проверьте, насколько хорошо "перетекают" получаемые на выходе лица по мере движения от вектора $z_1$ к вектору $z_2$.

**В-третьих, добавить больше реализованных идей из перечисленных в семинаре статей.** Мы не пробовали `Gradient Clipping` в `WGAN` (хотя сказали, что он должен быть хуже, чем `Gradient Penalty`), не добавляли спектральную нормализацию в генератор, не подбирали толком константы...

### 5.1. GAN Readlist

Статьи:
1. [Generative Adversarial Networks](https://papers.nips.cc/paper/5423-generative-adversarial-nets.pdf) (GAN)
2. [Conditional GAN](https://arxiv.org/abs/1411.1784) (cGAN)
3. [Unsupervised representation learning with deep convolutional generative adversarial networks](https://arxiv.org/pdf/1511.06434) (DCGAN)
4. [Image-to-Image Translation with Conditional Adversarial Networks](https://arxiv.org/abs/1611.07004) [Pix2Pix]
5. [Wasserstein GAN](https://arxiv.org/abs/1701.07875) (Wasserstein GAN)
6. [Improved Training of Wasserstein GANs](https://arxiv.org/abs/1704.00028) (Wasserstein GAN + Gradient Penalty)
7. [Spectral Normalization for Generative Adversarial Networks](https://arxiv.org/abs/1802.05957) (SNGAN)
8. [Self-Attention Generative Adverarial Networks](https://arxiv.org/pdf/1805.08318) (SAGAN)
9. [Large Scale GAN Training for High Fidelity Natural Image Synthesis](https://arxiv.org/abs/1809.11096) (BigGAN)
10. [A Style-Based Generator Architecture for Generative Adversarial Networks](https://arxiv.org/abs/1812.04948) (StyleGAN)

Репозитории и код:
1. [PyTorch DCGAN example](https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html) (основа для этого семинара)
2. [PyTorch-GAN](https://github.com/eriklindernoren/PyTorch-GAN) (простые реализации множества статей по GAN)
3. [BigGAN](https://github.com/eriklindernoren/PyTorch-GAN) (реализация огромного числа фич для GAN)
4. [BigGAN TF Hub Demo](https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/biggan_generation_with_tf_hub.ipynb) (ноутбук с предобученным BigGAN)
5. [How to Train a GAN? Tips and tricks to make GANs work](https://github.com/soumith/ganhacks)

![interpolation](resources/interpolation.png)